In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
import seaborn as sns
import os
import re
import spacy
import ast
from datetime import datetime, date
nlp = spacy.load('en')
from collections import Counter
%matplotlib inline

In [343]:
list_df = pd.read_csv('./archive/listings.csv')
cal_df = pd.read_csv('./archive/calendar.csv')
review_df = pd.read_csv('./archive/reviews.csv')

We want to determine what features a listing must have to generate the highest price point. Features examined will be neighbourhood, amenities included, and rating

- What style of accommodation  allows for highest revenue?
- Can we predict price using categorical and quantitative values?
- What effect does distance from city center have on price?

The listings dataset contains the fields of interest in analyzing the dataset. The calendar and reviews csv files are not needed for the purpose of this project. The following cell analyzed the size and data types contained in the file.

In [280]:
print('Shape:', list_df.shape)
print('Data types column count:')
print(list_df.dtypes.value_counts())

Shape: (3818, 92)
Data types column count:
object     62
float64    17
int64      13
dtype: int64


The dataset contains entries for 3818 listings with 92 variables per listing. These variables are represented by a mix of object, float64, and int64 data types. This indicates that a mix of quantitative and categorical variables can be found for each listing, complicating our analysis of the data.

Now that the types of data contained in the dataset are known, it is important to analyze missing values in the dataset. This will aid in determining which columns might not be useful in our analysis due to a lack of entries.

In [130]:
#Count the number of missing values and then sort in descending order
#pd.set_option("max_rows", None)
list_df.isna().sum().sort_values(ascending=False)

license                             3818
square_feet                         3721
monthly_price                       2301
security_deposit                    1952
weekly_price                        1809
notes                               1606
neighborhood_overview               1032
cleaning_fee                        1030
transit                              934
host_about                           859
host_acceptance_rate                 773
review_scores_accuracy               658
review_scores_checkin                658
review_scores_value                  656
review_scores_location               655
review_scores_cleanliness            653
review_scores_communication          651
review_scores_rating                 647
reviews_per_month                    627
first_review                         627
last_review                          627
space                                569
host_response_time                   523
host_response_rate                   523
neighbourhood   

The missing values in some of these columns will need to be dealt with to accurately analyze the data. Certain columns containing missing values are not of interest, contain over 50 % missing values without an explanation, or are categorical variables containing a written description that is not within the scope of our analysis. These columns dropped completely (license, notes, neighborhood_overview, transit, host_about, first_review, last_review, space, any of the url vairables, summary, host_location). Monthly and weekly price variables will be dropped considering the price variable encompasses these values and is more universally listed in the dataset. Security_deposit, cleaning_fee, and extra_people are all columns that contain fee information where an absent value is likely an indication of a zero $ fee and the missing value will be replaced with 0. All other columns with missing values will have the missing entries filled with mean or mode values.

### Data Transformations

The quantitative and categorical columns of interest are separated.

In [360]:
quantitative = ['host_since', 'host_response_rate', 'host_acceptance_rate',
       'host_total_listings_count', 'accommodates', 'price', 'availability_365',
       'bathrooms', 'bedrooms', 'beds', 'square_feet', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'number_of_reviews','review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'reviews_per_month']

categorical = ['host_verifications', 'amenities','host_response_time','host_is_superhost',  
       'host_identity_verified', 'property_type', 'room_type', 'bed_type', 'instant_bookable', 'cancellation_policy', 
       'require_guest_profile_picture', 'require_guest_phone_verification', 'neighbourhood_group_cleansed',
       'requires_license', 'is_location_exact', 'has_availability']

# 'neighbourhood_cleansed', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'host_has_profile_pic',

In [361]:
drop_list = [i for i in list_df.columns if i not in quantitative+categorical]

In [362]:
test_df = list_df.drop(drop_list, axis=1)    

Certain columns that belong to each group must be transformed for accurate analysis. Columns in the date_cols category are in the format of "YYYY-MM-DD" as strings and must be transformed into a numerical count of the days since the indicated date. This quantitative variable should allow for meaningful information to be extracted as the larger the number, the more time has passed. Columns in the value_cols category are given as prices including dollar signs in a string format. These must be converted to floats to be analyzed. The rest of the quantitative variables (in num_cols) are already in float or int format and need no conversion.

list_cols indicates the categorical columns that contain a string list of values. These entries must be converted to a list and then each possible amenity or verification will be made into a single column with a 1 or 0 value for whether a listing contains that amenity/verification. This is necessary as creating dummy variables from the list entries would create dummy columns for each separate possible list, regardless of the amenities/verifications contained within that list.

In [363]:
date_cols = ['host_since']
value_cols = ['host_response_rate', 'host_acceptance_rate',
              'price', 'extra_people', 'security_deposit', 'cleaning_fee']

#columns in which a nan value represents a zero $ fee
nan_zero_cols = ['cleaning_fee', 'extra_people'] 

#columns in which nan values will be filled with the mode rather than the mean as a float value would not be realistic
nan_mode_cols = ['host_total_listings_count', 'accommodates',
               'bathrooms', 'bedrooms', 'beds','minimum_nights',
               'maximum_nights', 
               'availability_365', 'number_of_reviews'] 

num_cols = [i for i in quantitative if i not in date_cols+value_cols]

list_cols = ['host_verifications', 'amenities']
cat_cols = [i for i in categorical if i not in list_cols]

In [364]:
#reformat_date takes in a list of column names containing entries in a date format and replaces them with 
#a number of days since the date using the datetime library
def reformat_date(df, date_vars):
    for col in date_vars:
        for i in df[col].index:
            if not pd.isnull(df[col][i]):
                delta = date.today() - date.fromisoformat(df[col][i])
                df.loc[i, col] = delta.days
        df.loc[:, col] = df[col].astype(float)
        
    return df

test_df = reformat_date(test_df, date_cols)

In [365]:
#reformat_price takes in a list of column names containing entries in a price string format and replaces them with 
#a float value
def reformat_price(df, value_vars):
    for col in value_vars:
        for i in df[col].index:
            if not pd.isnull(df[col][i]):
                df.loc[i, col] = re.sub("[$,%]", "", df[col][i])
        df.loc[:, col] = df[col].astype(float)
    
    return df

test_df = reformat_price(test_df, value_cols)

In [366]:
#reformat_list takes in a list of column names containing entries in a string list format and replaces them with a 
#basic list that is then used to create dummy columns for each possible list entry value
def reformat_list(df, list_vars):
    for col in list_vars:
        new_col = []
        for i in df[col].index:
            if not pd.isnull(df[col][i]):
                if '{' in df[col][i]:
                    new_col.append(df[col][i].strip('{}').split(','))
                else:
                    new_col.append(ast.literal_eval(df[col][i]))
            else:
                new_col.append([])
        df = df.drop(col, axis=1)
        df = df.assign(temp=pd.Series(new_col).values)
        df = pd.concat([df.drop('temp', axis=1), pd.get_dummies(df['temp'].apply(pd.Series).stack()).sum(level=0)], axis=1)
    return df

test_df = reformat_list(test_df, list_cols)

In [367]:
#fill_null fills in all null values in the quantitative columns with the mean of the column, the mode, or a zero value
def fill_null(df, mode_cols, zero_cols):
    num_vars = df.select_dtypes(include=['float', 'int']).columns
    for col in num_vars:
        if col in zero_cols:
            df.loc[:, col].fillna(0, inplace=True)
        elif col in mode_cols:
            df.loc[:, col].fillna((df[col].mode()[0]), inplace=True)
        else:
            df.loc[:, col].fillna((df[col].mean()), inplace=True)
    return df

test_df = fill_null(test_df, nan_mode_cols, nan_zero_cols)

In [368]:
def dummy_columns(df, cat_vars):
    for col in cat_vars:
        try:
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], 
                 prefix=col, prefix_sep='_', dummy_na=False, drop_first=True)], axis=1)
        except:
            continue
    return df

test_df = dummy_columns(test_df, cat_cols)

In [411]:
y = test_df['price']
X = test_df.drop(['price', ''], axis=1)    

In [413]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=28) 

lm_model = LinearRegression(normalize=True) # Instantiate
lm_model.fit(X_train, y_train) #Fit

#Predict and score the model
y_test_preds = lm_model.predict(X_test) 
"The r-squared score for your model was {} on {} values.".format(r2_score(y_test, y_test_preds), len(y_test))

'The r-squared score for your model was 0.7467956772104349 on 1146 values.'

In [414]:
def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

#Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

#A quick look at the top results
coef_df

,est_int,coefs,abs_coefs
86,property_type_Boat,105.089654,105.089654
101,room_type_Shared room,-61.571629,61.571629
61,"""Washer / Dryer""",42.195288,42.195288
38,weibo,36.055053,36.055053
98,property_type_Treehouse,35.982781,35.982781
35,photographer,32.021869,32.021869
119,neighbourhood_group_cleansed_Magnolia,29.799803,29.799803
100,room_type_Private room,-29.382227,29.382227
89,property_type_Camper/RV,27.592858,27.592858
34,phone,26.665739,26.665739


In [369]:
new_col = []
for i in test_df['price'].index:
    if not pd.isnull(test_df['price'][i]):
        new_col.append(test_df['price'][i]*test_df['availability_365'][i])
    else:
        new_col.append([])

test_df = test_df.assign(revenue=pd.Series(new_col).values)

"new_col = []\nfor i in test_df['price'].index:\n    if not pd.isnull(test_df['price'][i]):\n        new_col.append(test_df['price'][i]*test_df['availability_365'][i])\n    else:\n        new_col.append([])\n\ntest_df = test_df.assign(revenue=pd.Series(new_col).values)"

In [385]:
#test_df.loc[:, test_df.columns.str.startswith('property')]
for col in df.columns[pd.Series(df.columns).str.startswith('property')]:
    col = []
    for i in df[col].index:
        if df[col][i] == 1:
            
    

Index(['property_type_Bed & Breakfast', 'property_type_Boat',
       'property_type_Bungalow', 'property_type_Cabin',
       'property_type_Camper/RV', 'property_type_Chalet',
       'property_type_Condominium', 'property_type_Dorm',
       'property_type_House', 'property_type_Loft', 'property_type_Other',
       'property_type_Tent', 'property_type_Townhouse',
       'property_type_Treehouse', 'property_type_Yurt'],
      dtype='object')

In [429]:
test_df['price'][1795]

135.0

In [433]:
price_dict = {}
for col in test_df.columns[pd.Series(test_df.columns).str.startswith('property')]:
    price_dict[col] = 0
    for i in test_df[col].index:
        if test_df[col][i] == 1:
            price_dict[col] += (test_df['price'][i] * (365 - test_df['availability_365'][i]))
    

In [435]:
price_dict

{'property_type_Bed & Breakfast': 383453.0,
 'property_type_Boat': 73105.0,
 'property_type_Bungalow': 141210.0,
 'property_type_Cabin': 232274.0,
 'property_type_Camper/RV': 138724.0,
 'property_type_Chalet': 8640.0,
 'property_type_Condominium': 1976192.0,
 'property_type_Dorm': 12395.0,
 'property_type_House': 27892463.0,
 'property_type_Loft': 700289.0,
 'property_type_Other': 89439.0,
 'property_type_Tent': 48260.0,
 'property_type_Townhouse': 2065544.0,
 'property_type_Treehouse': 67290.0,
 'property_type_Yurt': 18800.0}

In [214]:
pd.get_dummies(az_df.apply(pd.Series).stack()).sum(level=0)

,amex,email,facebook,google,jumio,kba,linkedin,manual_offline,manual_online,phone,photographer,reviews,sent_id,weibo
0,0,1,0,0,0,1,0,0,0,1,0,1,0,0
1,0,1,1,0,1,0,1,0,0,1,0,1,0,0
2,0,1,0,1,1,0,0,0,0,1,0,1,0,0
3,0,1,1,0,1,0,0,0,0,1,0,1,0,0
4,0,1,1,0,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,0,1,0,0,1,0,1,0,0,1,0,1,0,0
3814,0,1,1,0,1,0,0,0,0,1,0,1,0,0
3815,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3816,0,1,1,0,0,1,0,0,0,1,0,1,0,0


In [215]:
amen_ls = list_df[categorical]['amenities'][0].strip('{}').split(',')
for x in amen_ls:
    x = x.strip('""')
amen_ls

['TV',
 '"Cable TV"',
 'Internet',
 '"Wireless Internet"',
 '"Air Conditioning"',
 'Kitchen',
 'Heating',
 '"Family/Kid Friendly"',
 'Washer',
 'Dryer']

In [221]:
list_df['host_verifications'][4].strip('{}')

"['email', 'phone', 'facebook', 'reviews', 'kba']"

In [234]:
others = [i for i in categorical if i not in list_cols]


,host_location,host_response_time,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,property_type,room_type,bed_type,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,"Seattle, Washington, United States",within a few hours,f,t,t,Queen Anne,Apartment,Entire home/apt,Real Bed,f,moderate,f,f
1,"Seattle, Washington, United States",within an hour,t,t,t,Queen Anne,Apartment,Entire home/apt,Real Bed,f,strict,t,t
2,"Seattle, Washington, United States",within a few hours,f,t,t,Queen Anne,House,Entire home/apt,Real Bed,f,strict,f,f
3,"Seattle, Washington, United States",NaN,f,t,t,Queen Anne,Apartment,Entire home/apt,Real Bed,f,flexible,f,f
4,"Seattle, Washington, United States",within an hour,f,t,t,Queen Anne,House,Entire home/apt,Real Bed,f,strict,f,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,US,within a few hours,f,t,t,Other neighborhoods,House,Entire home/apt,Real Bed,f,strict,f,f
3814,"Seattle, Washington, United States",within an hour,f,t,t,Capitol Hill,Apartment,Entire home/apt,Real Bed,f,moderate,f,f
3815,US,NaN,f,t,f,Rainier Valley,House,Entire home/apt,Real Bed,f,moderate,f,f
3816,"Tacoma, Washington, United States",within an hour,f,t,t,Capitol Hill,Condominium,Entire home/apt,Real Bed,f,moderate,f,f


In [228]:
list_df[quantitative]

,host_since,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,square_feet,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,2011-08-11,96%,100%,3.0,3.0,4,1.0,1.0,1.0,NaN,...,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,4.07
1,2013-02-21,98%,100%,6.0,6.0,4,1.0,1.0,1.0,NaN,...,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,1.48
2,2014-06-12,67%,100%,2.0,2.0,11,4.5,5.0,7.0,NaN,...,2014-07-30,2015-09-03,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.15
3,2013-11-06,NaN,NaN,1.0,1.0,3,1.0,0.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-11-29,100%,NaN,2.0,2.0,6,2.0,3.0,3.0,NaN,...,2012-07-10,2015-10-24,92.0,9.0,9.0,10.0,10.0,9.0,9.0,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,2015-04-13,99%,100%,354.0,354.0,6,2.0,3.0,3.0,NaN,...,2015-09-27,2015-09-27,80.0,8.0,10.0,4.0,8.0,10.0,8.0,0.30
3814,2015-10-14,100%,100%,1.0,1.0,4,1.0,1.0,2.0,NaN,...,2015-12-18,2015-12-24,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.00
3815,2015-12-30,NaN,NaN,1.0,1.0,2,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3816,2015-01-03,100%,NaN,1.0,1.0,2,1.0,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
exp_df = list_df[cat_cols]
list_df[list_cols]['host_verifications'][0]

"['email', 'phone', 'reviews', 'kba']"

In [299]:
exp_df['host_verifications'][0]

"['email', 'phone', 'reviews', 'kba']"

In [341]:
target = list_df[list_cols]
target['host_verifications'][0]

"['email', 'phone', 'reviews', 'kba']"

In [342]:
for i in target['host_verifications'].index:
    if not pd.isnull(target['host_verifications'][i]):
        if '{' in target['host_verifications'][i]:
            print('oops')
            target.loc[i, 'host_verifications'] = target['host_verifications'][i].strip('{}').split(',')
        target.loc[i, 'host_verifications'] = ast.literal_eval(target['host_verifications'][i])

/Users/justinharrison/thesis/env/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/justinharrison/thesis/env/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [350]:
for x in target['host_verifications']:
    print(type(x) )

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

In [345]:
target['host_verifications'][i]

['email', 'phone', 'reviews', 'kba']

In [32]:
def clean_data(df, drop_var):
    
    df = df.dropna(subset=['price'], axis=0)
    
    for col in date_cols:
        for i in df[col].index:
            if not pd.isnull(df[col][i]):
                delta = date.today() - date.fromisoformat(df[col][i])
                df.loc[i, col] = delta.days
        df.loc[:, col] = df[col].astype(float)

    for col in value_cols:
        for i in df[col].index:
            if not pd.isnull(df[col][i]):
                df.loc[i, col] = re.sub("[$,%]", "", df[col][i])
        df.loc[:, col] = df[col].astype(float)

    for col in list_cols:
        for i in df[col].index:
            if not pd.isnull(df[col][i]):
                if '{' in df[col][i]:
                    df[col][i] = df[col][i].strip('{}').split(',')
                else:
                    df[col][i] = ast.literal_eval(df[col][i])
                
    for col in cat_cols:
        try:
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], 
                 prefix=col, prefix_sep='_', dummy_na=False, drop_first=True)], axis=1)
        except:
            df = df.drop(col, axis=1)
            continue

            
    num_vars = df.select_dtypes(include=['float', 'int']).columns
    
    
    for col in num_vars:
        if col in nan_zero_cols:
            df.loc[:, col].fillna(0, inplace=True)
        else:
            df.loc[:, col].fillna((df[col].mean()), inplace=True)
    
    y = df['price']
    df = df.drop(drop_var, axis=1)    
    X = df

    return X, y
    
#Use the function to create X and y
drop_list = ['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 
             'experiences_offered', 'host_about','neighborhood_overview', 'space', 'price', 'description', 
             'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
             'host_id', 'host_url', 'host_name', 'square_feet', 'host_thumbnail_url', 'host_picture_url',
             'zipcode', 'market', 'smart_location', 'country', 'latitude', 'longitude', 'weekly_price',
             'monthly_price', 'calendar_updated', 'calendar_last_scraped', 'license', 'jurisdiction_names', 'street',
             'neighbourhood', 'neighbourhood_group_cleansed', 'host_neighbourhood', 'host_location'] 

X, y = clean_data(list_df, drop_list)    

/Users/justinharrison/thesis/env/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/justinharrison/thesis/env/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
